In [5]:
import sys
sys.path.append('../classes') 

In [8]:
sys.path.append('../models') 

In [13]:
from transformers import CLIPVisionModel

In [14]:
from genomic_plip_model import GenomicPLIPModel

In [15]:
import torch

In [16]:
original_model = CLIPVisionModel.from_pretrained("../plip/")
genomic_plip = GenomicPLIPModel(original_model)
genomic_plip.load_state_dict(torch.load('../models/genomic_plip.pth'))
# genomic_plip.eval()

<All keys matched successfully>

In [18]:
genomic_plip.eval()  # Switch to evaluation mode if needed

# Save the model's state_dict
model_save_path = '../models/genomic_plip_model.bin'  # Path where you want to save the model
torch.save(genomic_plip.state_dict(), model_save_path)

In [19]:
import json

In [24]:
plip_config =original_model.config

In [28]:
plip_config_dict = plip_config.to_dict()

In [30]:
# plip_config_dict

In [32]:
modifications = {
    "additional_head": {
        "type": "Linear",
        "in_features": 768,
        "out_features": 512
    },
    "fc_layer": {
        "type": "Linear",
        "in_features": 4,
        "out_features": 512
    }
}

In [33]:
# Combine the base model config with the modifications
genomic_plip_config = {
    "model_type": "CLIPVisionModel+GenomicPLIP",
    "base_model_config": plip_config_dict,
    "modifications": modifications,
    "weights_source": "Hugging Face/your-custom-model",
    "description": "This model is an extension of CLIPVisionModel with a custom head for Genomic PLIP tasks."
}

In [35]:
with open('../../genomic_plip_hug/genomic_plip_config.json', 'w') as fp:
    json.dump(genomic_plip_config, fp, indent=4)

In [36]:
{
    "model_lineage": {
        "original": "CLIP",
        "intermediate": "PLIP",
        "current": "Genomic PLIP"
    },
    "current_model_config": {
        "model_type": "GenomicPLIPModel",
        "description": "Genomic PLIP, an extension of the PLIP model with additional functionality for genomic analysis tasks.",
        "base_model": "PLIP",
        "modifications": {
            "vision_projection": {
                "type": "Linear",
                "in_features": 768,
                "out_features": 512
            },
            "fc_layer": {
                "type": "Linear",
                "in_features": 4,
                "out_features": 512
            }
        }
    },
    "reference_configs": {
        "CLIP": "path/to/clip/config.json",
        "PLIP": "path/to/plip/config.json"
    }
}


{'model_lineage': {'original': 'CLIP',
  'intermediate': 'PLIP',
  'current': 'Genomic PLIP'},
 'current_model_config': {'model_type': 'GenomicPLIPModel',
  'description': 'Genomic PLIP, an extension of the PLIP model with additional functionality for genomic analysis tasks.',
  'base_model': 'PLIP',
  'modifications': {'vision_projection': {'type': 'Linear',
    'in_features': 768,
    'out_features': 512},
   'fc_layer': {'type': 'Linear', 'in_features': 4, 'out_features': 512}}},
 'reference_configs': {'CLIP': 'path/to/clip/config.json',
  'PLIP': 'path/to/plip/config.json'}}

In [40]:
with open('../../genomic_plip_hug/plip_config.json', 'r') as file:
    config = json.load(file)

In [41]:
config

{'_commit_hash': None,
 'architectures': ['CLIPModel'],
 'initializer_factor': 1.0,
 'logit_scale_init_value': 2.6592,
 'model_type': 'clip',
 'projection_dim': 512,
 'text_config': {'_name_or_path': '',
  'add_cross_attention': False,
  'architectures': None,
  'attention_dropout': 0.0,
  'bad_words_ids': None,
  'begin_suppress_tokens': None,
  'bos_token_id': 0,
  'chunk_size_feed_forward': 0,
  'cross_attention_hidden_size': None,
  'decoder_start_token_id': None,
  'diversity_penalty': 0.0,
  'do_sample': False,
  'dropout': 0.0,
  'early_stopping': False,
  'encoder_no_repeat_ngram_size': 0,
  'eos_token_id': 2,
  'exponential_decay_length_penalty': None,
  'finetuning_task': None,
  'forced_bos_token_id': None,
  'forced_eos_token_id': None,
  'hidden_act': 'quick_gelu',
  'hidden_size': 512,
  'id2label': {'0': 'LABEL_0', '1': 'LABEL_1'},
  'initializer_factor': 1.0,
  'initializer_range': 0.02,
  'intermediate_size': 2048,
  'is_decoder': False,
  'is_encoder_decoder': False,


In [42]:
config['model_type'] = 'genomic_plip'
config['architectures'] = ['GenomicPLIPModel']

# Since you modified the vision projection, update these settings
config['vision_config']['projection_dim'] = 512  # Updated to match your vision_projection layer
config['vision_config']['hidden_size'] = 768  # Reflects the input size to your vision_projection

# Add custom fields for new layers or significant architecture changes
# Assuming you want to detail the new fully connected layer architecture
config['genomic_config'] = {
    'fc_layer_input': 4,
    'fc_layer_output': 512
}


In [44]:
# config

In [46]:
with open('../../genomic_plip_hug/config.json', 'w') as file:
    json.dump(config, file, indent=4)